In [127]:
# consumir o nlp e com base nos dados, utilizar um label encoder para a classe do comentario do aluno e utilziar xgboost com o resultado

In [128]:
# para tratar datasets desbalanceados
!pip install imbalanced-learn

In [129]:
# para utilizar no lemmatizer com um dicionário em pt-BR
!python -m spacy download pt_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 52.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [130]:
import pandas as pd
import joblib
import nltk
import re
import string
import unicodedata
import spacy
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import RandomOverSampler
import warnings

In [131]:
pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

In [132]:
# NTLK
nltk.download('stopwords')
nltk.download('punkt')

# Spacy
nlp_spacy_ptbr = spacy.load("pt_core_news_sm")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [133]:
df = pd.read_csv('/content/processado_base_full.csv', sep=';')
df.head()

,INSTITUICAO_ENSINO_ALUNO,NOME,IDADE_ALUNO,ANOS_PM,FASE_TURMA,PONTO_VIRADA,INDE,INDE_CONCEITO,PEDRA,DESTAQUE_IEG,DESTAQUE_IDA,DESTAQUE_IPV,IAA,IEG,IPS,IDA,IPP,IPV,IAN,ANO,FASE,TURMA,SINALIZADOR_INGRESSANTE,REC_EQUIPE_1,REC_EQUIPE_2,REC_EQUIPE_3,REC_EQUIPE_4,NIVEL_IDEAL,DEFASAGEM,ANO_INGRESSO,BOLSISTA,CG,CF,CT,NOTA_PORT,NOTA_MAT,NOTA_ING,QTD_AVAL,REC_AVA_1,REC_AVA_2,REC_AVA_3,REC_AVA_4,INDICADO_BOLSA
0,Escola Pública,ALUNO-1,11.0,2.0,2H,Não,7.883752,B,Ametista,Seu destaque em 2020: A sua boa entrega das li...,Ponto a melhorar em 2021: Empenhar-se mais nas...,Seu destaque em 2020: A sua boa integração aos...,8.50002,8.7,7.500,7.000000,5.9375,7.750000,10.0,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Escola Pública,ALUNO-3,12.0,2.0,3H,Não,7.856390,B,Ametista,Seu destaque em 2020: A sua boa entrega das li...,Ponto a melhorar em 2021: Empenhar-se mais nas...,Seu destaque em 2020: A sua boa integração aos...,7.91667,8.9,7.500,5.500000,8.1250,8.111113,10.0,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Escola Pública,ALUNO-4,10.0,2.0,1D,Não,5.076252,D,Quartzo,Ponto a melhorar em 2021: Melhorar a sua entre...,Ponto a melhorar em 2021: Empenhar-se mais nas...,Ponto a melhorar em 2021: Integrar-se mais aos...,8.00002,4.1,6.875,0.000000,7.1875,7.750000,5.0,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Escola Pública,ALUNO-5,10.0,1.0,2M,Não,8.077085,B,Ametista,Seu destaque em 2020: A sua boa entrega das li...,Seu destaque em 2020: As suas boas notas na Pa...,Seu destaque em 2020: A sua boa integração aos...,7.50002,8.0,7.500,7.500000,8.4375,8.166665,10.0,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Rede Decisão/União,ALUNO-8,14.0,3.0,4L,Sim,8.381391,B,Ametista,Seu destaque em 2020: A sua boa entrega das li...,Ponto a melhorar em 2021: Empenhar-se mais nas...,Seu destaque em 2020: A sua boa integração aos...,8.33334,9.9,4.375,7.333333,8.7500,8.944450,10.0,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [134]:
df['PONTO_VIRADA'].value_counts()

PONTO_VIRADA
Não    1932
Sim     315
Name: count, dtype: int64

In [135]:
df.query('PONTO_VIRADA == "Sim"').head(10)

,INSTITUICAO_ENSINO_ALUNO,NOME,IDADE_ALUNO,ANOS_PM,FASE_TURMA,PONTO_VIRADA,INDE,INDE_CONCEITO,PEDRA,DESTAQUE_IEG,DESTAQUE_IDA,DESTAQUE_IPV,IAA,IEG,IPS,IDA,IPP,IPV,IAN,ANO,FASE,TURMA,SINALIZADOR_INGRESSANTE,REC_EQUIPE_1,REC_EQUIPE_2,REC_EQUIPE_3,REC_EQUIPE_4,NIVEL_IDEAL,DEFASAGEM,ANO_INGRESSO,BOLSISTA,CG,CF,CT,NOTA_PORT,NOTA_MAT,NOTA_ING,QTD_AVAL,REC_AVA_1,REC_AVA_2,REC_AVA_3,REC_AVA_4,INDICADO_BOLSA
4,Rede Decisão/União,ALUNO-8,14.0,3.0,4L,Sim,8.381391,B,Ametista,Seu destaque em 2020: A sua boa entrega das li...,Ponto a melhorar em 2021: Empenhar-se mais nas...,Seu destaque em 2020: A sua boa integração aos...,8.33334,9.9,4.375,7.333333,8.750000,8.944450,10.0,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Escola Pública,ALUNO-11,8.0,0.0,0A,Sim,9.712504,A,Topázio,Seu destaque em 2020: A sua boa entrega das li...,Seu destaque em 2020: As suas boas notas na Pa...,Seu destaque em 2020: A sua boa integração aos...,9.00002,10.0,8.125,10.000000,10.000000,10.000010,10.0,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,Rede Decisão/União,ALUNO-28,13.0,2.0,3J,Sim,9.354726,A,Topázio,Seu destaque em 2020: A sua boa entrega das li...,Seu destaque em 2020: As suas boas notas na Pa...,Seu destaque em 2020: A sua boa integração aos...,9.58335,9.6,6.875,9.000000,10.000000,9.944453,10.0,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,Rede Decisão/União,ALUNO-55,17.0,3.0,6C,Sim,6.878125,C,Ágata,Seu destaque em 2020: A sua boa entrega das li...,Ponto a melhorar em 2021: Empenhar-se mais nas...,Ponto a melhorar em 2021: Integrar-se mais aos...,0.00000,9.0,7.500,6.333333,8.281250,8.666667,5.0,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,Rede Decisão/União,ALUNO-75,11.0,1.0,3L,Sim,9.687504,A,Topázio,Seu destaque em 2020: A sua boa entrega das li...,Seu destaque em 2020: As suas boas notas na Pa...,Seu destaque em 2020: A sua boa integração aos...,10.00002,10.0,10.000,9.000000,9.375000,9.750010,10.0,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,Escola Pública,ALUNO-81,15.0,1.0,4L,Sim,7.386391,B,Ametista,Seu destaque em 2020: A sua boa entrega das li...,Ponto a melhorar em 2021: Empenhar-se mais nas...,Ponto a melhorar em 2021: Integrar-se mais aos...,5.41667,8.8,7.500,6.000000,8.125000,9.111120,5.0,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42,Rede Decisão/União,ALUNO-83,14.0,4.0,4B,Sim,8.155003,B,Ametista,Seu destaque em 2020: A sua boa entrega das li...,Ponto a melhorar em 2021: Empenhar-se mais nas...,Seu destaque em 2020: A sua boa integração aos...,9.16668,9.4,5.625,5.333333,8.958333,9.166677,10.0,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,Rede Decisão/União,ALUNO-87,14.0,4.0,4L,Sim,8.188613,B,Ametista,Seu destaque em 2020: A sua boa entrega das li...,Ponto a melhorar em 2021: Empenhar-se mais nas...,Seu destaque em 2020: A sua boa integração aos...,8.33334,9.2,5.625,6.666667,8.750000,8.722227,10.0,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,Rede Decisão/União,ALUNO-88,11.0,2.0,2D,Sim,9.040420,A,Topázio,Seu destaque em 2020: A sua boa entrega das li...,Seu destaque em 2020: As suas boas notas na Pa...,Seu destaque em 2020: A sua boa integração aos...,10.00002,9.4,7.500,9.500000,5.937500,9.583340,10.0,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53,Escola Pública,ALUNO-96,18.0,1.0,7A,Sim,6.432501,C,Ágata,Ponto a melhorar em 2021: Melhorar a sua entre...,Ponto a melhorar em 2021: Empenhar-se mais nas...,Ponto a melhorar em 2021: Integrar-se mais aos...,9.16668,6.1,6.875,3.000000,7.500000,8.791665,5.0,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [136]:
df_ml = df[['PONTO_VIRADA', 'DESTAQUE_IEG',	'DESTAQUE_IDA',	'DESTAQUE_IPV',	'IAA',	'IEG',	'IPS',	'IDA', 	'IPP',	'IPV',	'IAN', 'REC_EQUIPE_1', 'REC_EQUIPE_2', 'REC_EQUIPE_3', 'REC_EQUIPE_4', 'REC_AVA_1', 'REC_AVA_2', 'REC_AVA_3', 'REC_AVA_4']]
df_ml.head()

,PONTO_VIRADA,DESTAQUE_IEG,DESTAQUE_IDA,DESTAQUE_IPV,IAA,IEG,IPS,IDA,IPP,IPV,IAN,REC_EQUIPE_1,REC_EQUIPE_2,REC_EQUIPE_3,REC_EQUIPE_4,REC_AVA_1,REC_AVA_2,REC_AVA_3,REC_AVA_4
0,Não,Seu destaque em 2020: A sua boa entrega das li...,Ponto a melhorar em 2021: Empenhar-se mais nas...,Seu destaque em 2020: A sua boa integração aos...,8.50002,8.7,7.500,7.000000,5.9375,7.750000,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Não,Seu destaque em 2020: A sua boa entrega das li...,Ponto a melhorar em 2021: Empenhar-se mais nas...,Seu destaque em 2020: A sua boa integração aos...,7.91667,8.9,7.500,5.500000,8.1250,8.111113,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Não,Ponto a melhorar em 2021: Melhorar a sua entre...,Ponto a melhorar em 2021: Empenhar-se mais nas...,Ponto a melhorar em 2021: Integrar-se mais aos...,8.00002,4.1,6.875,0.000000,7.1875,7.750000,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Não,Seu destaque em 2020: A sua boa entrega das li...,Seu destaque em 2020: As suas boas notas na Pa...,Seu destaque em 2020: A sua boa integração aos...,7.50002,8.0,7.500,7.500000,8.4375,8.166665,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Sim,Seu destaque em 2020: A sua boa entrega das li...,Ponto a melhorar em 2021: Empenhar-se mais nas...,Seu destaque em 2020: A sua boa integração aos...,8.33334,9.9,4.375,7.333333,8.7500,8.944450,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [137]:
modelo = joblib.load('/content/model.pkl')
vect = joblib.load("vect.pkl")

In [138]:
def normalize_accents(text):
    return unicodedata.normalize("NFKD", text).encode("ASCII", "ignore").decode("utf-8")

def remove_punctuation(text):
    punctuations = string.punctuation
    table = str.maketrans({key: " " for key in punctuations})
    text = text.translate(table)
    return text

def normalize_str(text):
    text = text.lower()
    text = remove_punctuation(text)
    text = normalize_accents(text)
    text = re.sub(re.compile(r" +"), " ",text)
    return " ".join([w for w in text.split()])

def tokenizer(text):
    stop_words = stopwords.words("portuguese")
    if isinstance(text, str):
        text = normalize_str(text)
        text = "".join([w for w in text if not w.isdigit()])
        text = word_tokenize(text)
        text = [x for x in text if x not in stop_words]
        text = [y for y in text if len(y) > 2]
        return " ".join([t for t in text])
    else:
        return None

def lemmatize(text):
    doc = nlp_spacy_ptbr(text)
    return " ".join([token.lemma_ for token in doc])

def predict_sentiment(texto):
    texto_teste = tokenizer(texto)
    texto_lematizado = lemmatize(texto_teste)
    text_vect = vect.transform([texto_lematizado])
    return modelo.predict(text_vect)

In [139]:
df_ml['DESTAQUE_IDA'] = df_ml['DESTAQUE_IDA'].apply(lambda x: predict_sentiment(x) if pd.notnull(x) else 'NA')
df_ml['DESTAQUE_IEG'] = df_ml['DESTAQUE_IEG'].apply(lambda x: predict_sentiment(x) if pd.notnull(x) else 'NA')
df_ml['DESTAQUE_IPV'] = df_ml['DESTAQUE_IPV'].apply(lambda x: predict_sentiment(x) if pd.notnull(x) else 'NA')
df_ml['REC_AVA_1'] = df_ml['REC_AVA_1'].apply(lambda x: predict_sentiment(x) if pd.notnull(x) else 'NA')
df_ml['REC_AVA_2'] = df_ml['REC_AVA_2'].apply(lambda x: predict_sentiment(x) if pd.notnull(x) else 'NA')
df_ml['REC_AVA_3'] = df_ml['REC_AVA_3'].apply(lambda x: predict_sentiment(x) if pd.notnull(x) else 'NA')
df_ml['REC_AVA_4'] = df_ml['REC_AVA_4'].apply(lambda x: predict_sentiment(x) if pd.notnull(x) else 'NA')
df_ml['REC_EQUIPE_1'] = df_ml['REC_EQUIPE_1'].apply(lambda x: predict_sentiment(x) if pd.notnull(x) else 'NA')
df_ml['REC_EQUIPE_2'] = df_ml['REC_EQUIPE_2'].apply(lambda x: predict_sentiment(x) if pd.notnull(x) else 'NA')
df_ml['REC_EQUIPE_3'] = df_ml['REC_EQUIPE_3'].apply(lambda x: predict_sentiment(x) if pd.notnull(x) else 'NA')
df_ml['REC_EQUIPE_4'] = df_ml['REC_EQUIPE_4'].apply(lambda x: predict_sentiment(x) if pd.notnull(x) else 'NA')

In [140]:
df_ml

,PONTO_VIRADA,DESTAQUE_IEG,DESTAQUE_IDA,DESTAQUE_IPV,IAA,IEG,IPS,IDA,IPP,IPV,IAN,REC_EQUIPE_1,REC_EQUIPE_2,REC_EQUIPE_3,REC_EQUIPE_4,REC_AVA_1,REC_AVA_2,REC_AVA_3,REC_AVA_4
0,Não,[positivo],[positivo],[positivo],8.50002,8.700000,7.500,7.000000,5.93750,7.750000,10.0,NA,NA,NA,NA,NA,NA,NA,NA
1,Não,[positivo],[positivo],[positivo],7.91667,8.900000,7.500,5.500000,8.12500,8.111113,10.0,NA,NA,NA,NA,NA,NA,NA,NA
2,Não,[positivo],[positivo],[positivo],8.00002,4.100000,6.875,0.000000,7.18750,7.750000,5.0,NA,NA,NA,NA,NA,NA,NA,NA
3,Não,[positivo],[positivo],[positivo],7.50002,8.000000,7.500,7.500000,8.43750,8.166665,10.0,NA,NA,NA,NA,NA,NA,NA,NA
4,Sim,[positivo],[positivo],[positivo],8.33334,9.900000,4.375,7.333333,8.75000,8.944450,10.0,NA,NA,NA,NA,NA,NA,NA,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2268,Não,[positivo],[positivo],[positivo],7.91668,9.666667,7.500,7.750000,6.25000,7.500000,10.0,NA,NA,NA,NA,[negativo],[positivo],[negativo],NA
2269,Não,[positivo],[positivo],[positivo],7.41668,6.470435,6.250,2.000000,6.25000,7.444443,5.0,NA,NA,NA,NA,[negativo],[positivo],[negativo],NA
2270,Sim,[positivo],[positivo],[positivo],9.00002,9.031839,7.500,9.000000,5.62500,9.083340,5.0,NA,NA,NA,NA,[positivo],[positivo],NA,NA
2271,Não,[positivo],[positivo],[positivo],8.33334,9.178030,5.625,5.900000,6.71875,5.833328,5.0,NA,NA,NA,NA,[negativo],[negativo],[positivo],[positivo]


In [141]:
clone = df_ml.copy()

In [142]:
clone.fillna(-1, inplace=True)

In [143]:
clone.head()

,PONTO_VIRADA,DESTAQUE_IEG,DESTAQUE_IDA,DESTAQUE_IPV,IAA,IEG,IPS,IDA,IPP,IPV,IAN,REC_EQUIPE_1,REC_EQUIPE_2,REC_EQUIPE_3,REC_EQUIPE_4,REC_AVA_1,REC_AVA_2,REC_AVA_3,REC_AVA_4
0,Não,[positivo],[positivo],[positivo],8.50002,8.7,7.500,7.000000,5.9375,7.750000,10.0,NA,NA,NA,NA,NA,NA,NA,NA
1,Não,[positivo],[positivo],[positivo],7.91667,8.9,7.500,5.500000,8.1250,8.111113,10.0,NA,NA,NA,NA,NA,NA,NA,NA
2,Não,[positivo],[positivo],[positivo],8.00002,4.1,6.875,0.000000,7.1875,7.750000,5.0,NA,NA,NA,NA,NA,NA,NA,NA
3,Não,[positivo],[positivo],[positivo],7.50002,8.0,7.500,7.500000,8.4375,8.166665,10.0,NA,NA,NA,NA,NA,NA,NA,NA
4,Sim,[positivo],[positivo],[positivo],8.33334,9.9,4.375,7.333333,8.7500,8.944450,10.0,NA,NA,NA,NA,NA,NA,NA,NA


In [144]:
cols = ['PONTO_VIRADA', 'DESTAQUE_IDA', 'DESTAQUE_IEG', 'DESTAQUE_IPV', 'REC_AVA_1', 'REC_AVA_2', 'REC_AVA_3', 'REC_AVA_4', 'REC_EQUIPE_1', 'REC_EQUIPE_2', 'REC_EQUIPE_3', 'REC_EQUIPE_4']
label_encoder = LabelEncoder()
df_to_encode = clone.astype(str)

for col in cols:
    clone[col] = label_encoder.fit_transform(df_to_encode[col])

clone

,PONTO_VIRADA,DESTAQUE_IEG,DESTAQUE_IDA,DESTAQUE_IPV,IAA,IEG,IPS,IDA,IPP,IPV,IAN,REC_EQUIPE_1,REC_EQUIPE_2,REC_EQUIPE_3,REC_EQUIPE_4,REC_AVA_1,REC_AVA_2,REC_AVA_3,REC_AVA_4
0,1,1,1,1,8.50002,8.700000,7.500,7.000000,5.93750,7.750000,10.0,0,0,0,0,0,0,0,0
1,1,1,1,1,7.91667,8.900000,7.500,5.500000,8.12500,8.111113,10.0,0,0,0,0,0,0,0,0
2,1,1,1,1,8.00002,4.100000,6.875,0.000000,7.18750,7.750000,5.0,0,0,0,0,0,0,0,0
3,1,1,1,1,7.50002,8.000000,7.500,7.500000,8.43750,8.166665,10.0,0,0,0,0,0,0,0,0
4,2,1,1,1,8.33334,9.900000,4.375,7.333333,8.75000,8.944450,10.0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2268,1,1,1,1,7.91668,9.666667,7.500,7.750000,6.25000,7.500000,10.0,0,0,0,0,1,2,1,0
2269,1,1,1,1,7.41668,6.470435,6.250,2.000000,6.25000,7.444443,5.0,0,0,0,0,1,2,1,0
2270,2,1,1,1,9.00002,9.031839,7.500,9.000000,5.62500,9.083340,5.0,0,0,0,0,2,2,0,0
2271,1,1,1,1,8.33334,9.178030,5.625,5.900000,6.71875,5.833328,5.0,0,0,0,0,1,1,2,2


In [145]:
X = clone.drop('PONTO_VIRADA', axis=1)
y = clone['PONTO_VIRADA']

ros = RandomOverSampler(random_state=777)
X_ros, y_ros = ros.fit_resample(X, y)
df_ros = pd.concat([X_ros, y_ros], axis=1)
df_ros

,DESTAQUE_IEG,DESTAQUE_IDA,DESTAQUE_IPV,IAA,IEG,IPS,IDA,IPP,IPV,IAN,REC_EQUIPE_1,REC_EQUIPE_2,REC_EQUIPE_3,REC_EQUIPE_4,REC_AVA_1,REC_AVA_2,REC_AVA_3,REC_AVA_4,PONTO_VIRADA
0,1,1,1,8.50002,8.700000,7.500,7.000000,5.937500,7.750000,10.0,0,0,0,0,0,0,0,0,1
1,1,1,1,7.91667,8.900000,7.500,5.500000,8.125000,8.111113,10.0,0,0,0,0,0,0,0,0,1
2,1,1,1,8.00002,4.100000,6.875,0.000000,7.187500,7.750000,5.0,0,0,0,0,0,0,0,0,1
3,1,1,1,7.50002,8.000000,7.500,7.500000,8.437500,8.166665,10.0,0,0,0,0,0,0,0,0,1
4,1,1,1,8.33334,9.900000,4.375,7.333333,8.750000,8.944450,10.0,0,0,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5791,1,1,1,9.00002,9.100000,7.500,9.000000,8.541667,8.833337,10.0,0,0,0,0,0,0,0,0,2
5792,1,1,1,10.00002,9.552042,7.500,8.466667,5.625000,8.500000,10.0,0,0,0,0,2,1,0,0,2
5793,1,1,1,9.50002,10.000000,8.750,9.500000,6.666667,9.611120,5.0,0,0,0,0,2,2,2,0,2
5794,1,1,1,9.00002,8.500000,6.250,6.500000,5.000000,9.083335,10.0,0,0,0,0,0,0,0,0,2


In [146]:
X = df_ros.drop('PONTO_VIRADA', axis=1)
y = df_ros['PONTO_VIRADA']

In [147]:
y.value_counts()

PONTO_VIRADA
1    1932
2    1932
0    1932
Name: count, dtype: int64

In [148]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=777)

In [149]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [150]:
def melhores_hiperparamentros_xgb(X_train, y_train):
    param_grid = {
      'learning_rate':[0.01, 0.05, 0.2],
      'n_estimators':[50, 100, 200, 500],
      'max_depth': [1, 2, 5, 8]
    }
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=777)

    grid = GridSearchCV(
      GradientBoostingClassifier(),
      param_grid = param_grid,
      scoring = 'f1',
      cv = cv,
      n_jobs= -1
    )

    # seleciona os melhores hiperparâmetros com base nos dados de treino
    grid.fit(X_train, y_train)
    xgb_params = grid.best_params_

    print("Melhores hiperparâmetros para XGB:", xgb_params)

    return xgb_params

In [151]:
params = melhores_hiperparamentros_xgb(X_train, y_train)
params

# Melhores hiperparâmetros para XGB: {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 50}
# {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 50}

Melhores hiperparâmetros para XGB: {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 50}


{'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 50}

In [153]:
kfold = KFold(n_splits=40, shuffle=True)
xgb = GradientBoostingClassifier(learning_rate=0.01, max_depth=5, n_estimators=200)
xgb.fit(X_train, y_train)
xgb_result = cross_val_score(xgb, X_train, y_train, cv=kfold)

In [154]:
y_pred = xgb.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       386
           1       1.00      1.00      1.00       387
           2       1.00      1.00      1.00       387

    accuracy                           1.00      1160
   macro avg       1.00      1.00      1.00      1160
weighted avg       1.00      1.00      1.00      1160



In [155]:
xgb_acuracia = accuracy_score(y_true=y_test, y_pred=y_pred) * 100
xgb_acuracia

100.0

In [ ]:
# TODO: algo de errado nao está certo... ta dando 100% de precisão :(